# Semantic Segmentation with on-the-fly Data Augmentation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
import os
from gm_networks import unet_model
from utils import center_crop, normalize_data, create_dataset_from_one_image

In [ ]:
# Load the initial input/output images. Normalize the input images to [0, 1] 
x_initial_train = normalize_data(plt.imread('train/101-INPUT.jpg'))
y_initial_train = plt.imread('train/101-OUTPUT-GT.png')
x_initial_train = center_crop(x_initial_train, (2560, 2560))
y_initial_train = center_crop(y_initial_train, (2560, 2560))

In [ ]:
# Model/training hyperparameters
INPUT_SHAPE = (256, 256, 3)
INITIAL_FILTERS = 32
BATCH_SIZE = 4
EPOCHS = 200

In [ ]:
model = unet_model(input_shape=INPUT_SHAPE, initial_filters=INITIAL_FILTERS)

In [ ]:
# Create a dataset with independent patches
MODE = 'independent_patches'
x_train, y_train = create_dataset_from_one_image(x_initial_train, y_initial_train, INPUT_SHAPE[0], mode=MODE)

In [ ]:
# Create a dataset with overlapped patches
MODE = 'overlapped_patches'
x_train, y_train = create_dataset_from_one_image(x_initial_train, y_initial_train, INPUT_SHAPE[0], mode=MODE, patches_step=256)

In [ ]:
# Create a dataset with random patches
MODE = 'random_patches'
x_train, y_train = create_dataset_from_one_image(x_initial_train, y_initial_train, INPUT_SHAPE[0], mode=MODE, new_images_number=81)

In [ ]:
print(x_train.shape)

In [ ]:
# Create dictionaries with the alterations to be used for the data augmentation operations for inputs and outputs
""""
NOTE: It is important to use exact the same values for the parameters in both dictionaries. 
The reason why we create 2 dictionaries instead of a common one is because  we want to add the 
preprocessing function for the output masks (this function sets all the pixel values of the mask to 0 or 1. 
While the initial ground truth images are binary, after the data augmentation operations, such as rotations and 
shifts, pixels with intermediate values are appeared due to interpolation)
"""
x_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect') 

# TODO: Post a question in StackOverflow to check why nan values are created, when I use the 'normalize_data' 
# function (from utils.py) as a preprocessing_function in x_datagen_args

y_datagen_args = x_datagen_args.copy()
y_datagen_args['preprocessing_function'] = lambda x: np.where(x>0, 1, 0).astype(x.dtype)

In [ ]:
# Instatiate the generators
x_datagen = ImageDataGenerator(**x_datagen_args)
y_datagen = ImageDataGenerator(**y_datagen_args)

# Setting the same seed number for both generators leads to the same random alterations for both generators
seed = 19
x_datagen.fit(x_train, augment=True, seed=seed)
y_datagen.fit(y_train, augment=True, seed=seed)

x_generator_train = x_datagen.flow(x_train, seed=seed, batch_size=BATCH_SIZE)
y_generator_train = y_datagen.flow(y_train, seed=seed, batch_size=BATCH_SIZE)

train_generator = zip(x_generator_train, y_generator_train)

In [ ]:
training_images_number = x_train.shape[0]
steps_per_epoch = training_images_number // BATCH_SIZE

# Define the model and train
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=EPOCHS)

In [ ]:
# save the model and history of the training (loss, accuracy)
model_path = f'trained_models/cropped_001/unet_inputsize{INPUT_SHAPE[0]}_initialfilters{INITIAL_FILTERS}_{MODE}_initialimagesnumber{x_train.shape[0]}_optimizerAdam_lr{"{:0.4f}".format(model.optimizer.lr.value().numpy())}_epochs{EPOCHS}' #TODO: loss and metric'
model.save(model_path)

In [ ]:
np.save(os.path.join(model_path, 'history'), history.history)

In [ ]:
type(model.optimizer.lr.value().numpy())

In [ ]:
"{:0.5f}".format(model.optimizer.lr.value().numpy())